In [39]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore') 
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from feature_engine.encoding import OneHotEncoder

from feature_engine.discretisation import EqualWidthDiscretiser
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

In [19]:
pip install feature-engine

Note: you may need to restart the kernel to use updated packages.


In [20]:
mark_data = pd.read_csv('/kaggle/input/predict-conversion-in-digital-marketing-dataset/digital_marketing_campaign_dataset.csv')

In [21]:
mark_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CustomerID           8000 non-null   int64  
 1   Age                  8000 non-null   int64  
 2   Gender               8000 non-null   object 
 3   Income               8000 non-null   int64  
 4   CampaignChannel      8000 non-null   object 
 5   CampaignType         8000 non-null   object 
 6   AdSpend              8000 non-null   float64
 7   ClickThroughRate     8000 non-null   float64
 8   ConversionRate       8000 non-null   float64
 9   WebsiteVisits        8000 non-null   int64  
 10  PagesPerVisit        8000 non-null   float64
 11  TimeOnSite           8000 non-null   float64
 12  SocialShares         8000 non-null   int64  
 13  EmailOpens           8000 non-null   int64  
 14  EmailClicks          8000 non-null   int64  
 15  PreviousPurchases    8000 non-null   i

In [22]:
mark_data.head()

,CustomerID,Age,Gender,Income,CampaignChannel,CampaignType,AdSpend,ClickThroughRate,ConversionRate,WebsiteVisits,PagesPerVisit,TimeOnSite,SocialShares,EmailOpens,EmailClicks,PreviousPurchases,LoyaltyPoints,AdvertisingPlatform,AdvertisingTool,Conversion
0,8000,56,Female,136912,Social Media,Awareness,6497.870068,0.043919,0.088031,0,2.399017,7.396803,19,6,9,4,688,IsConfid,ToolConfid,1
1,8001,69,Male,41760,Email,Retention,3898.668606,0.155725,0.182725,42,2.917138,5.352549,5,2,7,2,3459,IsConfid,ToolConfid,1
2,8002,46,Female,88456,PPC,Awareness,1546.429596,0.277490,0.076423,2,8.223619,13.794901,0,11,2,8,2337,IsConfid,ToolConfid,1
3,8003,32,Female,44085,PPC,Conversion,539.525936,0.137611,0.088004,47,4.540939,14.688363,89,2,2,0,2463,IsConfid,ToolConfid,1
4,8004,60,Female,83964,PPC,Conversion,1678.043573,0.252851,0.109940,0,2.046847,13.993370,6,6,6,8,4345,IsConfid,ToolConfid,1


In [23]:
mark_data.describe(include='all')

,CustomerID,Age,Gender,Income,CampaignChannel,CampaignType,AdSpend,ClickThroughRate,ConversionRate,WebsiteVisits,PagesPerVisit,TimeOnSite,SocialShares,EmailOpens,EmailClicks,PreviousPurchases,LoyaltyPoints,AdvertisingPlatform,AdvertisingTool,Conversion
count,8000.00000,8000.000000,8000,8000.000000,8000,8000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000,8000,8000.000000
unique,NaN,NaN,2,NaN,5,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,NaN
top,NaN,NaN,Female,NaN,Referral,Conversion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IsConfid,ToolConfid,NaN
freq,NaN,NaN,4839,NaN,1719,2077,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8000,8000,NaN
mean,11999.50000,43.625500,NaN,84664.196750,NaN,NaN,5000.944830,0.154829,0.104389,24.751625,5.549299,7.727718,49.799750,9.476875,4.467375,4.485500,2490.268500,NaN,NaN,0.876500
std,2309.54541,14.902785,NaN,37580.387945,NaN,NaN,2838.038153,0.084007,0.054878,14.312269,2.607358,4.228218,28.901165,5.711111,2.856564,2.888093,1429.527162,NaN,NaN,0.329031
min,8000.00000,18.000000,NaN,20014.000000,NaN,NaN,100.054813,0.010005,0.010018,0.000000,1.000428,0.501669,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.000000
25%,9999.75000,31.000000,NaN,51744.500000,NaN,NaN,2523.221165,0.082635,0.056410,13.000000,3.302479,4.068340,25.000000,5.000000,2.000000,2.000000,1254.750000,NaN,NaN,1.000000
50%,11999.50000,43.000000,NaN,84926.500000,NaN,NaN,5013.440044,0.154505,0.104046,25.000000,5.534257,7.682956,50.000000,9.000000,4.000000,4.000000,2497.000000,NaN,NaN,1.000000
75%,13999.25000,56.000000,NaN,116815.750000,NaN,NaN,7407.989369,0.228207,0.152077,37.000000,7.835756,11.481468,75.000000,14.000000,7.000000,7.000000,3702.250000,NaN,NaN,1.000000


In [24]:
mark_data.nunique()

CustomerID             8000
Age                      52
Gender                    2
Income                 7789
CampaignChannel           5
CampaignType              4
AdSpend                8000
ClickThroughRate       8000
ConversionRate         8000
WebsiteVisits            50
PagesPerVisit          8000
TimeOnSite             8000
SocialShares            100
EmailOpens               20
EmailClicks              10
PreviousPurchases        10
LoyaltyPoints          3983
AdvertisingPlatform       1
AdvertisingTool           1
Conversion                2
dtype: int64

In [25]:
mark_data=mark_data.drop(columns=['AdvertisingPlatform','AdvertisingTool','CustomerID'])

In [26]:
gender_mapping={'Male':0,'Female':1}
mark_data['Gender']=mark_data['Gender'].map(gender_mapping)

In [27]:
mark_data.head()

,Age,Gender,Income,CampaignChannel,CampaignType,AdSpend,ClickThroughRate,ConversionRate,WebsiteVisits,PagesPerVisit,TimeOnSite,SocialShares,EmailOpens,EmailClicks,PreviousPurchases,LoyaltyPoints,Conversion
0,56,1,136912,Social Media,Awareness,6497.870068,0.043919,0.088031,0,2.399017,7.396803,19,6,9,4,688,1
1,69,0,41760,Email,Retention,3898.668606,0.155725,0.182725,42,2.917138,5.352549,5,2,7,2,3459,1
2,46,1,88456,PPC,Awareness,1546.429596,0.277490,0.076423,2,8.223619,13.794901,0,11,2,8,2337,1
3,32,1,44085,PPC,Conversion,539.525936,0.137611,0.088004,47,4.540939,14.688363,89,2,2,0,2463,1
4,60,1,83964,PPC,Conversion,1678.043573,0.252851,0.109940,0,2.046847,13.993370,6,6,6,8,4345,1


In [28]:
X_train,X_test,y_train,y_test=train_test_split(
    mark_data.drop('Conversion',axis=1),mark_data['Conversion'],test_size=69,stratify=mark_data['Conversion']
)

In [29]:
X_train.shape,X_test.shape

((7931, 16), (69, 16))

In [30]:
y_train.shape,y_test.shape

((7931,), (69,))

In [40]:
ohe_enc = OneHotEncoder(
    top_categories=None,
    variables=['CampaignChannel','CampaignType'], 
    drop_last=False)
ohe_enc.fit(X_train)

OneHotEncoder(variables=['CampaignChannel', 'CampaignType'])

In [41]:
X_train=ohe_enc.transform(X_train)
X_test=ohe_enc.transform(X_test)

In [42]:
X_train.head()

,Age,Gender,Income,AdSpend,ClickThroughRate,ConversionRate,WebsiteVisits,PagesPerVisit,TimeOnSite,SocialShares,...,LoyaltyPoints,CampaignChannel_Social Media,CampaignChannel_Referral,CampaignChannel_PPC,CampaignChannel_SEO,CampaignChannel_Email,CampaignType_Conversion,CampaignType_Awareness,CampaignType_Consideration,CampaignType_Retention
3960,38,0,77572,769.500644,0.120787,0.142027,4,7.971614,9.266167,25,...,899,1,0,0,0,0,1,0,0,0
626,58,1,39332,5485.299394,0.150652,0.107087,37,7.102511,14.275181,88,...,4051,1,0,0,0,0,0,1,0,0
3446,22,1,81739,3486.527222,0.161145,0.080841,29,3.028056,13.646895,42,...,3441,0,1,0,0,0,1,0,0,0
1849,22,1,44260,103.409243,0.207781,0.080491,14,9.291673,2.026071,4,...,2676,0,0,1,0,0,0,0,1,0
6625,39,1,51126,5074.391218,0.200547,0.121601,48,2.123415,4.493555,42,...,3127,1,0,0,0,0,0,0,1,0


In [43]:
ewq_disc=EqualWidthDiscretiser(bins=5,variables=['Age','ClickThroughRate'])
ewq_disc.fit(X_train)

EqualWidthDiscretiser(bins=5, variables=['Age', 'ClickThroughRate'])

In [44]:
ewq_disc.binner_dict_

{'Age': [-inf, 28.2, 38.4, 48.599999999999994, 58.8, inf],
 'ClickThroughRate': [-inf,
  0.06799753556046156,
  0.1259902176061221,
  0.1839828996517827,
  0.24197558169744324,
  inf]}

In [45]:
X_train=ewq_disc.transform(X_train)
X_test=ewq_disc.transform(X_test)

In [46]:
scaler=StandardScaler()

In [47]:
for col in X_train.columns:
    X_train[col]=scaler.fit_transform(X_train[[col]])
    X_test[col]=scaler.transform(X_test[[col]])

In [50]:
log_reg=LogisticRegression(random_state=69)
log_reg.fit(X_train,y_train)

y_pred_log=log_reg.predict(X_test)

print(accuracy_score(y_test,y_pred_log))
print(roc_auc_score(y_test,log_reg.predict_log_proba(X_test)[:,1]))

0.8985507246376812
0.7166666666666667


In [51]:
rf_clf=RandomForestClassifier(random_state=69)
rf_clf.fit(X_train,y_train)
y_pred_rf=rf_clf.predict(X_test)

print(accuracy_score(y_test,y_pred_rf))
print(roc_auc_score(y_test,rf_clf.predict_proba(X_test)[:,1]))

0.8695652173913043
0.8138888888888889


In [53]:
adb=AdaBoostClassifier(random_state=69)
adb.fit(X_train,y_train)

y_pred_adb=adb.predict(X_test)

print(accuracy_score(y_test,y_pred_adb))
print(roc_auc_score(y_test,adb.predict_proba(X_test)[:,1]))

0.8985507246376812
0.8148148148148149
